# Startup Funding Regression Project

In [21]:
!pip install fuzzywuzzy[speedup]

In [695]:
import pandas as pd
import numpy as np
import unicodedata
import codecs
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from fuzzywuzzy import process, fuzz
from collections import Counter
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### For this project I decided to use kaggle´s Indian Startup database due to its rawness and the feature engineering needed to make ML ready.

In [696]:
df = pd.read_csv('indian_startup.csv')
df.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [697]:
df.loc[df['Investors Name'].str.contains('undisclosed', na=False, case=False), 'Investors Name']

69                                            Undisclosed
129                                           Undisclosed
253     Undisclosed Existing Investors As Well As The ...
425                                 Undisclosed Investors
432                                 Undisclosed Investors
                              ...                        
2883                                Undisclosed Investors
2922                                          Undisclosed
2976                                          Undisclosed
3008                                Undisclosed Investors
3026                                Undisclosed Investors
Name: Investors Name, Length: 146, dtype: object

### With a size of 30444 rows, it gives a decent sample size for a regression ML model. Another thing to note is all the 'object' type columns.

In [698]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2084 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


### First thing I look for are the NAN percentages, I immediately spot a column that becomes useless for this project.

df.isna().sum() / len(df) * 100

### Since there is no way of filling NANs and due to the nature of the column, I decide to drop it.

In [699]:
df = df.drop(columns='Remarks')

## I also drop null values in our 'Amount in USD' column since this will be our target, so we cant fill them.

In [700]:
df = df.dropna(subset='Amount in USD')

### Next thing I notice is that the 'Amount' column is of 'object' type, so I look into which classes are mixed there.

In [701]:
df['Amount in USD'].apply(type).value_counts()

Amount in USD
<class 'str'>    2084
Name: count, dtype: int64

### If we look into the column, we notice that the numbers are formatted the Indian way, so Python takes them as strings.

In [702]:
df['Amount in USD'].head(10)

0    20,00,00,000
1       80,48,394
2     1,83,58,860
3       30,00,000
4       18,00,000
5       90,00,000
6    15,00,00,000
7       60,00,000
8     7,00,00,000
9     5,00,00,000
Name: Amount in USD, dtype: object

### I decide to write a function that cleans the strings and converts them to float type.

In [703]:
def clean_amount(value):
    if isinstance(value, str):
        try:
            return float(value.replace(',', ''))
        except ValueError:
            return np.nan
    elif isinstance(value, (int, float)):
        return value
    else:
        return np.nan
    

df['Amount in USD'] = df['Amount in USD'].apply(clean_amount)

### Here I hit one of the main stonewalls in this project, the SubVertital column is too inconsistent to be a feature, due to it being more of a 'business description' value rather than a categorical subvertical.

In [704]:
df['SubVertical'].head(20)

0                                   E-learning
1                    App based shuttle service
2        Retailer of baby and toddler products
3                            Online Investment
4                  Embroiled Clothes For Women
5     Open-market, freight management platform
6                Online Food Delivery Platform
7                                     Agritech
8                                   Automobile
9                      Satellite Communication
10            Logistics Services and Solutions
11                Food Solutions For Corporate
12    Online Meat And Seafood Ordering Startup
13               Non-Banking Financial Company
14               Experience Discovery Platform
15             Real money based gaming startup
16              Online Eyewear Shopping Portal
17      Business and customer engagement tools
18             Men's Health and Wellness brand
19                                   Elearning
Name: SubVertical, dtype: object

### That incosistency makes it so there are 1942 different values in a 3044 long database, 63,7% unique values. But there is potential here to create a valuable feature.

In [705]:
df['SubVertical'].nunique()

1315

### What I decide to go for is to create a new column, which will be my feature, and fill it with 'X' to mark the uncategorized ones.

In [706]:
df['Clean SubVertical'] = 'X'

### To handle the NANs here, due to it being a SubVertical, I decide to fill in with the business´s Vertical. 

In [707]:
df['SubVertical'] = df['SubVertical'].fillna(df['Industry Vertical'])

### And here comes the 'tedious' part, I´ll manually look at the database and recognize keywords and patterns so I can create my own subset of subverticals and recategorize the startups.

In [708]:
df.loc[(df['Clean SubVertical'] == 'X') & (df['SubVertical'].str.contains(r'delivery service', regex=True, case=False, na=False)), ['Industry Vertical','SubVertical','Clean SubVertical']].tail(50)

,Industry Vertical,SubVertical,Clean SubVertical
32,Customer Service,Delivery Service,X
181,Consumer Internet,Online Organic Juice Delivery Service,X
1180,Logistics,On-Demand Delivery Service,X
1852,Consumer Internet,Hyperlocal Grocery Delivery Service,X
2175,Hyperlocal\\xc2\\xa0 Grocery Delivery Service,Hyperlocal\\xc2\\xa0 Grocery Delivery Service,X
2275,Hyperlocal healthcare product Delivery Service...,Hyperlocal healthcare product Delivery Service...,X
2345,Last Mile Delivery Service,Last Mile Delivery Service,X
2355,Online food ordering & Delivery service,Online food ordering & Delivery service,X
2356,Online food ordering & Delivery service,Online food ordering & Delivery service,X
2413,Hyperlocal Delivery Services,Hyperlocal Delivery Services,X


### After looking at the database, I noticed 2 scenarios:
1. A set of keywords, in which case I filter by that subset of words and replace with the new category.
2. Descriptions were there is no keyword, but the context itself gives away which category it falls into, in this case I manually recategorize each one using its index.

In [709]:
def classify(rows, label):
    df.loc[rows, 'Clean SubVertical'] = label

df.loc[df['SubVertical'].str.contains(r'health|medic|doctor|clinic|hospitals?\b', regex=True, case=False, na=False), 'Clean SubVertical'] = 'Health Related Services'
classify([21,28,71,99,117,151,154,161,237,304,242,1871], 'Health Related Services')

df.loc[df['SubVertical'].str.contains('diagnos|probio|cancer', case=False, na=False),'Clean SubVertical'] = 'Diagnostics & Medical Testing'

df.loc[df['SubVertical'].str.contains('pharma', case=False, na=False), 'Clean SubVertical'] = 'Pharmacy'

# Here I first filter by 'food' and label all of them as 'Food Related Services'.Then if they include 'food delivery' or other, I recategorize as 'Grocery / Food Delivery' 
df.loc[df['SubVertical'].str.contains('beverage|meat|food', case=False, na=False), 'Clean SubVertical'] = 'Food Related Services'
classify([6,11,12,56,59,66,53,126,130,1856],'Food Related Services')
df.loc[df['SubVertical'].str.contains('food delivery|food ordering|grocer|coffee', case=False, na=False), 'Clean SubVertical'] = 'Grocery / Food Delivery Services'
classify([132,190,181,249], 'Grocery / Food Delivery Services')

df.loc[df['SubVertical'].str.contains(r'rick|rental car|vehicle rental|car rent|carpool|bike rent|taxi|bicycle|drivers|online car| cab |cab |cabs|(?=.*\bvehicle\b)(?=.*\bbooking\b)', regex=True, case=False, na=False), 'Clean SubVertical'] = 'Transportation Rental Services'
classify([1,26,29,64,60,87,110,133,172,183,247,621,221,721,2198,2376,1569,1546,1481,1637,1130],'Transportation Rental Services')

df.loc[df['SubVertical'].str.contains(r'(?=.*\belectric\b)(?=.*\bmanufacturer\b)', regex=True, case=False, na=False), 'Clean SubVertical'] = 'Electric Vehicle Manufacturer'

classify([8,44,46,50,61,94,116,134,147,205,724,1023,2102,679,1258,2668,2642,2657,2216,2410,2576,1293,1751,628,2244,2193,2123],'Automotive Related Services')

classify([236,849,1180,1196,1492,2345,2530], 'Doorstep Delivery Services')

df.loc[df['SubVertical'].str.contains('logistic|supply chain|truck|cargo', case=False, na=False), 'Clean SubVertical'] = 'Logistic & Supply Chain Related Services'
classify([5,10,32,45,69,92,11,145,111,152,149,150,253], 'Logistic & Supply Chain Related Services')

df.loc[df['SubVertical'].str.contains('fitness|wellness| spa |lifestyle', case=False, na=False), 'Clean SubVertical'] = 'Fitness & Wellness Related Services'
classify([18,42,93,102,136,222,224,2098], 'Fitness & Wellness Related Services')

df.loc[df['SubVertical'].str.contains('baby', case=False, na=False), 'Clean SubVertical'] = 'Baby Supplies'
classify([2,159], 'Baby Supplies')

df.loc[df['SubVertical'].str.contains('fashion|linger|apparel', case=False, na=False), 'Clean SubVertical'] = 'Fashion Retail'
classify([4,72,105,113,1599], 'Fashion Retail')

df.loc[df['SubVertical'].str.contains('eyewear', case=False, na=False), 'Clean SubVertical'] = 'Eyewear Retail'
classify([16], 'Eyewear Retail')

df.loc[df['SubVertical'].str.contains(r'(?=.*\blearning\b)(?=.*\bplatform\b)', regex=True, case=False, na=False), 'Clean SubVertical'] = 'E-learning'
df.loc[df['SubVertical'].str.contains(r'(?=.*\blearning\b)(?=.*\bapp\b)', regex=True, case=False, na=False), 'Clean SubVertical'] = 'E-learning'
df.loc[df['SubVertical'].str.contains(r'(?=.*\bkids\b)(?=.*\blearning\b)', regex=True, case=False, na=False), 'Clean SubVertical'] = 'E-learning'
df.loc[df['SubVertical'].str.contains('online certi|elearning|e-learning|how-to', regex=True, case=False, na=False), 'Clean SubVertical'] = 'E-Learning'

df.loc[df['SubVertical'].str.contains('tutor|coach', regex=True, case=False, na=False), 'Clean SubVertical'] = 'Coach & Tutoring Services'

df.loc[df['SubVertical'].str.contains('education|ed-tech|school|skill|study', regex=True, case=False, na=False), 'Clean SubVertical'] = 'Education'
classify([30,38,67,137,57,2064,2028,1827,1682,854,418,412,406,384,269,2020,316,372,369,2524], 'Education')

df.loc[df['SubVertical'].str.contains('game|gaming', case=False, na=False), 'Clean SubVertical'] = 'Gaming'
classify([15,24,129,158], 'Gaming')

df.loc[df['SubVertical'].str.contains('agri-|agriculture|farmer', case=False, na=False), 'Clean SubVertical'] = 'Agritech'
classify([7,22,81,95], 'Agritech')

df.loc[df['SubVertical'].str.contains('artificial int', case=False, regex=True, na=False), 'Clean SubVertical'] = 'AI Related'
df.loc[df['SubVertical'].str.contains(r'\bAI\b', case=False, regex=True, na=False), 'Clean SubVertical'] = 'AI Related'
classify([80,65,51,141,177,178,182,254], 'AI Related')

df.loc[df['SubVertical'].str.contains('wealth|invest|stock|trading|capital|bitcoin', case=False, na=False), 'Clean SubVertical'] = 'Investment/Wealth Management'
classify([98,55,47,3,188,220,250,246,238], 'Investment/Wealth Management')

df.loc[df['SubVertical'].str.contains('lending|loan|financ|payment|wallet', case=False, na=False), 'Clean SubVertical'] = 'Banking/Payments/Financial Services'
classify([78,77,103,86,85,13,23,54,31,35,36,162,163,173,175,184,225,251,257,1296,495,698,1210,2147], 'Banking/Payments/Financial Services')
classify([118,127,138,142,223,230,215,841,798,819], 'Non Banking Financial Services')

df.loc[df['SubVertical'].str.contains('credit|fraud|score|risk|fin-|finte', case=False, na=False), 'Clean SubVertical'] = 'Fintech / Credit Scoring / Risk Analysis'

df.loc[df['SubVertical'].str.contains('ecommerce|e-commerce', case=False, na=False), 'Clean SubVertical'] = 'Ecommerce'
classify([131,128,124,123,49,96,27,169,176,191], 'Ecommerce')

df.loc[df['SubVertical'].str.contains('property|real estate|flat rental|residential rental|rental acco', case=False, na=False), 'Clean SubVertical'] = 'Real Estate'
classify([125,74], 'Real Estate')

df.loc[df['SubVertical'].str.contains('travel|hotel|trip|holiday|night|budget acc|hospitality', case=False, na=False), 'Clean SubVertical'] = 'Travel/Trip Related Services'
classify([121,101,170,743], 'Travel Related Services')
classify([146,97,75,2008], 'Wine & Beer')

df.loc[df['SubVertical'].str.contains('marketing', case=False, na=False), 'Clean SubVertical'] = 'Marketing'
classify([107,37,], 'Marketing')

df.loc[df['SubVertical'].str.contains('robotic|robot', case=False, na=False), 'Clean SubVertical'] = 'Robotics'

df.loc[df['SubVertical'].str.contains('video|music|movie', case=False, na=False), 'Clean SubVertical'] = 'Video Content & Streaming Platforms'

df.loc[df['SubVertical'].str.contains('insurance', case=False, na=False), 'Clean SubVertical'] = 'Insurance / InsurTech'

df.loc[df['SubVertical'].str.contains('communic', case=False, na=False), 'Clean SubVertical'] = 'Communication Platforms'

df.loc[df['SubVertical'].str.contains(r'media|news|entertainment|\bbooks?\b', regex=True,case=False, na=False), 'Clean SubVertical'] = 'Digital Media / Content'

df.loc[df['SubVertical'].str.contains('SaaS|Software', case=False, na=False),'Clean SubVertical'] = 'Software / SaaS'

df.loc[df['SubVertical'].str.contains('data analy|analy|data', case=False, na=False),'Clean SubVertical'] = 'Data Science/Analytics'

df.loc[df['SubVertical'].str.contains('spaces|budget room', case=False, na=False),'Clean SubVertical'] = 'Co-Working & Co-Living Spaces'

df.loc[df['SubVertical'].str.contains('career|job|hiring|networking|recruit', case=False, na=False),'Clean SubVertical'] = 'Career & Recruitment Services'

df.loc[df['SubVertical'].str.contains('fund', case=False, na=False),'Clean SubVertical'] = 'Crowdfunding / Fundraising Platforms'

df.loc[df['SubVertical'].str.contains('matchm|dating|match', case=False, na=False),'Clean SubVertical'] = 'Dating Platforms'

df.loc[df['SubVertical'].str.contains('home|household product', case=False, na=False),'Clean SubVertical'] = 'Home Related Services'

df.loc[df['SubVertical'].str.contains('cybersecurity|cyber security|security', case=False, na=False),'Clean SubVertical'] = 'Cybersecurity'

df.loc[df['SubVertical'].str.contains('tea |herbal', case=False, na=False),'Clean SubVertical'] = 'Tea Sales'

df.loc[df['SubVertical'].str.contains('social network|blogg|viral|picture', case=False, na=False),'Clean SubVertical'] = 'Social Media / Networks'

df.loc[df['SubVertical'].str.contains('beauty|salon', case=False, na=False),'Clean SubVertical'] = 'Beauty Related Services'

df.loc[df['SubVertical'].str.contains('waste|recycl', case=False, na=False),'Clean SubVertical'] = 'Waste/Recycling & Enviroment Related Services'

df.loc[df['SubVertical'].str.contains('business', case=False, na=False),'Clean SubVertical'] = 'Business Development'

df.loc[df['SubVertical'].str.contains('restaurant', case=False, na=False),'Clean SubVertical'] = 'Restaurant Tech & Services'

df.loc[df['SubVertical'].str.contains('pet', case=False, na=False),'Clean SubVertical'] = 'Pet Related Services'

df.loc[df['SubVertical'].str.contains('customer|b2c', case=False, na=False),'Clean SubVertical'] = 'Customer Support/Engagement'

df.loc[df['SubVertical'].str.contains(r'(?=.*\bonline\b)(?=.*\bfurniture\b)|furniture', regex=True, case=False, na=False),'Clean SubVertical'] = 'Furniture Retail & Rent'

df.loc[df['SubVertical'].str.contains(r'app dev|app man|\bapi\b', regex=True, case=False, na=False),'Clean SubVertical'] = 'App Development & Management' # esta categoria se puede ampliar

df.loc[df['SubVertical'].str.contains('solar|energy', case=False, na=False),'Clean SubVertical'] = 'Energy'

df.loc[df['SubVertical'].str.contains('cleani', case=False, na=False),'Clean SubVertical'] = 'Cleaning Services'

df.loc[df['SubVertical'].str.contains('hyperlocal|handyman|hyper-', case=False, na=False),'Clean SubVertical'] = 'Hyperlocal Services'

df.loc[df['SubVertical'].str.contains('jewel|luxury', case=False, na=False),'Clean SubVertical'] = 'Jewellery'

df.loc[df['SubVertical'].str.contains('print', case=False, na=False),'Clean SubVertical'] = '3D Printing Services'

df.loc[df['SubVertical'].str.contains('ticket', case=False, na=False),'Clean SubVertical'] = 'Online Ticketing / Booking Services'

df.loc[df['SubVertical'].str.contains('hygi', case=False, na=False),'Clean SubVertical'] = 'Personal Hygiene'

df.loc[df['SubVertical'].str.contains('photo', case=False, na=False),'Clean SubVertical'] = 'Photography Related Services'

df.loc[df['SubVertical'].str.contains('classified', case=False, na=False),'Clean SubVertical'] = 'Classifieds'

df.loc[df['SubVertical'].str.contains('comparison|coupon|loyalty|rewards', case=False, na=False),'Clean SubVertical'] = 'Price Comparison & Coupon Aggregators'

df.loc[df['SubVertical'].str.contains('publish', case=False, na=False),'Clean SubVertical'] = 'Publishing Services'

df.loc[df['SubVertical'].str.contains('mobile repair|gadgets repair|gadget|device repair', case=False, na=False),'Clean SubVertical'] = 'Gadget Repairing Services'

df.loc[df['SubVertical'].str.contains('wedding', case=False, na=False),'Clean SubVertical'] = 'Wedding Planning Services'

df.loc[df['SubVertical'].str.contains('employee|human resources|gamification|intelligent sales', case=False, na=False),'Clean SubVertical'] = 'Human Resources / Employee Training'

### To help visualize progress, I created a percentage and rows counter.

In [710]:
# USE THIS CELL ONLY FOR PARTIAL RUNS!!!
df.loc[df['Clean SubVertical'] == 'X', 'Clean SubVertical'] = df['SubVertical']

In [711]:
progress = (df['Clean SubVertical'] != 'X').mean() * 100
rows = (df['Clean SubVertical'] != 'X').sum()
left = len(df) - rows
values = df['Clean SubVertical'].nunique() - 1

print(f"{progress:.1f}% done")
print(f'{rows} rows done')
print(f'{left} rows left')
print(f'{values} categories')

100.0% done
2084 rows done
0 rows left
433 categories


### Next, I plan to do the same with the Vertical column, but due to it having way clearer categories, I´ll wait until after running the model and seeing the scores.

In [712]:
df['Industry Vertical'].value_counts()

Industry Vertical
Consumer Internet             589
Technology                    310
eCommerce                     126
Finance                        57
Healthcare                     44
                             ... 
Cloud Enterprise Software       1
Startup Analytics platform      1
Financial Markets Software      1
Hiring Analytics platform       1
Ecommerce Logistics             1
Name: count, Length: 596, dtype: int64

### The other main wall I hit is the 'Investor Name' column. This column is a set of strings written in inconsistent form, mentioning the investors.
### 2 problems here:
1. SInce the dataset was webscrapped, we got UNICODE phantom spaces ingrained in our values, which need to be addressed before processing.
2. The name of the investors is written differently (letter cases, spelling, spaced, separation symbols).
3. It is not a list, so each row is a string and we cant access the investors so the model can learn information about them.

In [713]:
df['Investors Name'].value_counts()

Investors Name
Undisclosed Investors                                    29
Undisclosed investors                                    24
Group of Angel Investors                                 14
Sequoia Capital                                          13
Kalaari Capital                                          13
                                                         ..
Jungle Ventures, Redbright Partners, Blume Ventures       1
Seedrs Crowd funding platform                             1
Kickstarter                                               1
Google Capital                                            1
Rajan Anandan, Shankar Mahadevan, Deep Kalra & Others     1
Name: count, Length: 1675, dtype: int64

In [714]:
df['Investors Name'].nunique()

1675

### To get a clean set of investors I go with 3 steps:
1. Clean UNICODE non-breaking spaces and standarize the strings so I can turn them into lists.
2. Remove redudants words like 'Capital' or 'India' that may mislead fuzzywuzzy when mapping the values.
3. Use fuzzywuzzy´s toke_set_ratio to get the list of unique investors and then map them.

In [715]:
# group|limited|ltd|inc|corp|corporation|company|co|management|

In [716]:
df['Investors Name'] = df['Investors Name'].fillna('Undisclosed')

def normalize_string(s):
    s = codecs.decode(s, 'unicode_escape')
    s = unicodedata.normalize('NFKC',s)
    s = s.replace('\xa0',' ')
    s = s.strip().lower()
    return s

df['Investors Name'] = df['Investors Name'].apply(normalize_string)

redundant_words = r'\b(india|capital|partners?|ventures?|holdings?|advisors?|associates?|investors?|accelerator)\b'

df['Investors List'] = df['Investors Name'].str.replace(' and ',' , ').str.replace(' & ',' , ').str.replace(redundant_words,'', regex=True).str.split(',').apply(lambda x: [i.strip() for i in x])

In [679]:
all_investors = sorted({i for sublist in df['Investors List'] for i in sublist})


canonical_names = {}
for inv in all_investors:
    if canonical_names:  
        result = process.extractOne(inv, canonical_names.keys(), scorer=fuzz.token_set_ratio)
        if result:  
            match, score = result
            if score >= 70:
                canonical_names[inv] = match
            else:
                canonical_names[inv] = inv
        else:  
            canonical_names[inv] = inv
    else:
        canonical_names[inv] = inv  


### After using fuzzywuzzy I take a look into the results to see if it is good enough for mapping the database.

In [717]:
df1 = pd.DataFrame(canonical_names.items(), columns=['name','canon'])
df1[df1['canon'].str.contains('sequoia')] # The '' represents a fund composed by only redundant words: the 'India Capital', so this mapping correclty identifies it.

,name,canon
1727,sequoia,sequoia
1728,sequoia global equities,sequoia
1729,sequoia .,sequoia


### After looking at fuzzywuzzy´s result dictionary and being ok with it, I proceed to map the database´s values and running a value_counts to see the distribution.

In [718]:
df['Investors List'] = df['Investors List'].apply(lambda lst: [canonical_names[i] for i in lst])

investors = [inv for sublist in df['Investors List'] for inv in sublist]
counts = Counter(investors)
pd.DataFrame(counts.items(), columns=['Investor','Count']).sort_values('Count', ascending=False)

,Investor,Count
18,3 undisclosed hnis,124
199,14 others,122
2,sequoia,117
92,angel,92
23,\xc2\xa0 accel,88
...,...,...
1354,sudhakar reddy,1
1353,fulcrum,1
1352,lantern,1
1351,startup brasil program,1


### In order to not overload the features and flood the model with a high number of dummies, I take the main investors and map the rest as 'Rare'.

In [719]:
counts_s = pd.Series(counts).sort_values(ascending=False)
counts_s[counts_s > 5]

3 undisclosed hnis           124
14 others                    122
sequoia                      117
angel                         92
\xc2\xa0 accel                88
                            ... 
ani technologies pvt. ltd      6
alok mittal                    6
capitalg                       6
\xc2\xa0 tracxn labs           6
highway                        6
Length: 138, dtype: int64

In [720]:
main_investors = counts_s[counts_s > 5]

df['Investors List'] = df['Investors List'].apply(lambda inv_list: [inv if inv in main_investors else 'Rare' for inv in inv_list])

In [721]:
investors = [inv for sublist in df['Investors List'] for inv in sublist]
new_counts = Counter(investors)
pd.DataFrame(new_counts.items(), columns=['Investor','Count']).sort_values('Count', ascending=False)

,Investor,Count
1,Rare,2087
10,3 undisclosed hnis,124
63,14 others,122
2,sequoia,117
38,angel,92
...,...,...
126,beenos,6
123,\xc2\xa0 tracxn labs,6
132,kanpur angels,6
131,multiple,6


### Since models cannot read lists, I use a MultiLabelBinarizer to include the investor dummies in the main df.

In [722]:
mlb = MultiLabelBinarizer()

investor_dummies = mlb.fit_transform(df['Investors List'])

investor_columns = pd.DataFrame(investor_dummies, columns=mlb.classes_, index=df.index)

df = pd.concat([df, investor_columns], axis=1)


### Since the signal given by a startup having more then one 'Rare' investor is lost in the dummies, I decided to create a 'Rare' investor count feature.

In [723]:
df['Rare Investor Count'] = df['Investors List'].apply(lambda inv_list: sum(1 for inv in inv_list if inv not in main_investors))

### FInally, I drop NANs before creating my testing and training sets.

In [724]:
df = df.dropna()

In [725]:
df.isna().sum() / len(df) * 100

Sr No                  0.0
Date dd/mm/yyyy        0.0
Startup Name           0.0
Industry Vertical      0.0
SubVertical            0.0
                      ... 
warburg pincus         0.0
y combinator           0.0
zishaan hayath         0.0
zodius                 0.0
Rare Investor Count    0.0
Length: 151, dtype: float64

### For this regression project, our sets are as follows:
1. Features: Industry Vertical, Industry SubVertical, Investors, Number of Investors, Investing type and Location.
2. Target: Amount of USD invested in the startup.

In [726]:
df.columns

Index(['Sr No', 'Date dd/mm/yyyy', 'Startup Name', 'Industry Vertical',
       'SubVertical', 'City  Location', 'Investors Name', 'InvestmentnType',
       'Amount in USD', 'Clean SubVertical',
       ...
       'unilazer', 'unitus', 'ventureast', 'vertex', 'vy', 'warburg pincus',
       'y combinator', 'zishaan hayath', 'zodius', 'Rare Investor Count'],
      dtype='object', length=151)

In [727]:
X = pd.get_dummies(df.drop(columns=['Sr No','Date dd/mm/yyyy','Startup Name','Amount in USD','SubVertical','Investors Name','Investors List']))
y = df['Amount in USD']
y = np.log1p(y) # applied a log1p transformation to the target to reduce skewness and stabilize variance.

random_state=42
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=random_state)

In [728]:
train = X_train.shape
test = X_test.shape

print(f'Train Set Shape: {train}')
print(f'Test Set Shape: {test}')

Train Set Shape: (1541, 1280)
Test Set Shape: (386, 1280)


### I decide to go with a RandomForestRegressor through a RandomizedSearchCV to find the best parameters.

In [729]:
rf = RandomForestRegressor(random_state=random_state)

param_grid = {
    'n_estimators': [100, 150, 200],     
    'max_depth': [5, 10, 15, 20, 25],       
    'min_samples_split': [2, 5, 10, 15],          
    'min_samples_leaf': [1, 2, 4, 6],             
    'max_features': ['sqrt', 'log2', 0.7, 0.8],   
    'bootstrap': [True, False]                   
}

rfs = RandomizedSearchCV(rf, 
                        param_distributions=param_grid, 
                        cv=5, 
                        n_iter=30,
                        random_state=random_state,
                        verbose=0,
                        scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_error', 'r2'],
                        refit='neg_root_mean_squared_error'
                        )

rfs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=30,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 15, 20, 25],
                                        'max_features': ['sqrt', 'log2', 0.7,
                                                         0.8],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [100, 150, 200]},
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_absolute_error', 'r2'])

In [730]:
params = rfs.best_params_
params

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 0.7,
 'max_depth': 25,
 'bootstrap': True}

### As the main objective, I´m looking for a r2 score of 0.7 - 0.8 at minimum and a difference of < 0.12 between train and test sets. Plus a RMSE as low as possible (1M - 10m ballpark).

| Model Variant                                                 | Train RMSE (USD)   | Train MAE (USD)   | Train R² | Test RMSE (USD)    | Test MAE (USD)    | Test R² |
|---------------------------------------------------------------|--------------------|-------------------|----------|--------------------|-------------------|---------|
| RAW                                                           | 130,806,916.70     | 16,484,788.21     | 0.66     | 88,331,235.87      | 16,065,519.24     | 0.59    |
| 64% SV + Investors (count > 10) + "Rare" count                | 123,453,752.46     | 14,810,789.99     | 0.74     | 63,296,122.16      | 12,121,546.42     | 0.59    |
| 64% SV + Investors (count > 2) + "Rare" count                 | 131,155,863.80     | 15,568,877.24     | 0.76     | 63,433,094.21      | 11,891,291.92     | 0.62    |
| 64% SV + Investors (count > 6) + "Rare" count                 | 130,341,585.24     | 15,588,298.96     | 0.77     | 62,737,896.19      | 11,636,428.40     | 0.63    |
| 64% SV + Investors (count > 4) + "Rare" count                 | 129,955,007.39     | 15,592,787.25     | 0.77     | 62,802,491.87      | 11,635,895.43     | 0.63    |
| 64% SV + Investors (count > 5) + "Rare" count                 | 130,504,567.74     | 15,615,101.39     | 0.76     | 62,779,673.14      | 11,581,088.10     | 0.63    |
| 64% SV + Investors (count > 5) + "Rare" count + clean UNICODE | 124,078,280.02     | 14,666,309.87     | 0.77     | 62,234,371.13      | 11,513,829.22     | 0.62    |
| 75% SV + Investors (count > 5) + "Rare" count + clean UNICODE | 124,786,704.10     | 14,822,827.95     | 0.77     | 61,799,312.10      | 11,459,385.31     | 0.62    |



In [731]:
train_pred = rfs.predict(X_train)
test_pred = rfs.predict(X_test)

trmse = np.sqrt(mean_squared_error(np.expm1(y_train), np.expm1(train_pred)))
tmae = mean_absolute_error(np.expm1(y_train), np.expm1(train_pred))
tr2 = r2_score(y_train, train_pred)

rmse = np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(test_pred)))
mae = mean_absolute_error(np.expm1(y_test), np.expm1(test_pred))
r2 = r2_score(y_test, test_pred)

print(f'75% SV + investors (count > 5) + "rare" count TRAIN SCORES:')
print(f'RMSE: {trmse:.2f}')
print(f'MAE: {tmae:.2f}')
print(f'R2: {tr2:.2f}')
print('----------------------')
print(f'75% SV + investor list (count > 5) + "rare" count TEST SCORES:')
print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'R2: {r2:.2f}')

75% SV + investors (count > 5) + "rare" count TRAIN SCORES:
RMSE: 124786704.10
MAE: 14822827.95
R2: 0.77
----------------------
75% SV + investor list (count > 5) + "rare" count TEST SCORES:
RMSE: 61799312.10
MAE: 11459385.31
R2: 0.62


### Finally I decided to take a look into the results of the predictions.

In [732]:
y_pred = rfs.predict(X_test)
true_y_pred = np.expm1(y_pred)

results = pd.DataFrame({
    'Actual': np.expm1(y_test),
    'Predicted': true_y_pred
})

results['Error'] = results['Actual'] - results['Predicted']
results['Actual'] = results['Actual'].map('{:,.0f}'.format)
results['Predicted'] = results['Predicted'].map('{:,.0f}'.format)
results['Error'] = results['Error'].map('{:,.0f}'.format)

results

,Actual,Predicted,Error
2167,"30,000,000","5,060,555","24,939,445"
2473,"5,000,000","7,450,912","-2,450,912"
2792,"3,000,000","4,360,094","-1,360,094"
1302,"1,000,000","244,099","755,901"
378,"3,800,000","9,337,528","-5,537,528"
...,...,...,...
714,"25,000,000","15,465,153","9,534,847"
945,"1,000,000","4,167,945","-3,167,945"
1616,"900,000","4,120,489","-3,220,489"
1363,"3,000,000","4,428,542","-1,428,542"
